In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =====

## 3. Building Multi-Stage Recommender Systems

Recommender Systems (RecSys) are the engine of the modern internet and the catalyst for human decisions. Building a recommendation system is challenging because it requires multiple stages (data preprocessing, offline training, item retrieval, filtering, ranking, ordering, etc.) to work together seamlessly and efficiently. The biggest challenges for new practitioners are the lack of understanding around what RecSys look like in the real world, and the gap between examples of simple models and a production-ready end-to-end recommender systems.

The figure below represents a four-stage recommender systems. This is more complex process than only training a single model and deploying it, and it is much more realistic and closer to what's happening in the real-world recommender production systems.

In this lab, we are going to showcase how we can deploy a four-stage recommender systems using Merlin Systems library easily on Triton Inference Server. Let's go over the concepts in the figure briefly.

- **Retrieval:** This is the step to narrow down millions of items into thousands of candidates. We are going to train a Two-Tower item retrieval model to retrieve the relevant top-K candidate items.
- **Filtering:** This step is to exclude the already interacted or undesirable items from the candidate items set or to apply business logic rules. Although this is an important step, for this example we skip this step.
- **Scoring:** This is also known as ranking. Here the retrieved and filtered candidate items are being scored. We are going to train a ranking model to be able to use at our scoring step.
- **Ordering:** At this stage, we can order the final set of items that we want to recommend to the user. Here, we’re able to align the output of the model with business needs, constraints, or criteria.

To learn more about the four-stage recommender systems, you can listen to Even Oldridge's [Moving Beyond Recommender Models talk at KDD'21](https://www.youtube.com/watch?v=5qjiY-kLwFY&list=PL65MqKWg6XcrdN4TJV0K1PdLhF_Uq-b43&index=8) and read more in this [blog post](https://eugeneyan.com/writing/system-design-for-discovery/).

**Learning Objectives of this notebook**

- Train a ranking and retriveal model with Merlin Models
- Export user query tower, user and item features, and item embedding
- Create feature store with Feast and register features

**Import Required Libraries**

In [2]:
import os

import glob
import cudf 
import pandas as pd
import numpy as np
import nvtabular as nvt
from nvtabular.ops import *
import gc

from merlin.schema.tags import Tags
import merlin.models.tf as mm
from merlin.io.dataset import Dataset

import tensorflow as tf

2022-08-17 21:46:36.359256: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-17 21:46:38.716773: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8080 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:0a:00.0, compute capability: 7.0


In [3]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
data_path = '/workspace/data/ecom/'
output_path = os.path.join(data_path,'processed_nvt')
output_path2 = os.path.join(data_path,'processed_filtered')
BASE_DIR = os.environ.get(
    "BASE_DIR", "/workspace/recsys_tutorial/"
)

Read processed parquet files as Dataset objects.

In [5]:
# train = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
# valid = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

# # define schema object
# schema = train.schema

**Filter out the negative rows**

Here, we will filter our datasets with NVTabular `Filter()` operator to select only positive interaction rows where `target==1` in the dataset. We do that because we want to use `negative sampling` technique when training our candidate retrieval and ranking models.

In [6]:
# inputs = schema.column_names
# outputs = inputs >> Filter(f=lambda df: df["target"] == 1)

In [7]:
# workflow2 = nvt.Workflow(outputs)

# workflow2.fit(train)

# workflow2.transform(train).to_parquet(
#     output_path=os.path.join(output_path2, "train")
# )

# workflow2.transform(valid).to_parquet(
#     output_path=os.path.join(output_path2, "valid")
# )

In [8]:
# workflow2.save(os.path.join(output_path2, "workflow2"))

**Read filtered parquet files as Dataset objects.**

In [9]:
train = Dataset(os.path.join(output_path2, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path2, "valid", "*.parquet"), part_size="500MB")

# define schema object
schema = train.schema

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


### Building and Training a Candidate Retrieval Model with Merlin Models

Industrial recommender systems have major tasks to accomplish that can be quite demanding. One requirement is to deliver a recommendation under the expected latency requirements (e.g., within milliseconds) to warrant a good user experience. That might require a significant amount of creativity and engineering. And the second consideration is that we might want to minimize infrastructure costs while solving the latency issue, which is yet another obstacle to overcome!

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions or billions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. 

Therefore, industrial recommender systems usually consists of candidate retrieval and ranking (scoring) stages. The candidate retrieval stage retrieves candidate items that are relevant to user interests, while the ranking stage sorts candidate items by user interests.

In this notebook, we start with the first stage of multi-stage recommender systems- the Candidate Retrieval. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are `Matrix Factorization (MF)`, which learns low-rank user and item embeddings, and the `Two-Tower architecture`, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output. Such models can be efficiently served by indexing the trained item embeddings into an Approximate Nearest Neighbors (ANN) engine and during inference scoring user embeddings over all indexed item embeddings within the engine. 

#### Two-Tower Model

We are going to train a Two-Tower model for item retrieval. A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately.

--- ADD IMAGE---

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling here and here), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [10]:
schema = train.schema
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

In [11]:
model_tt = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

In [12]:
%%time
model_tt.compile(
    optimizer="adam",
    run_eagerly=False,
    loss="categorical_crossentropy",
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)],
)
model_tt.fit(train, validation_data=valid, batch_size=1024 * 8, epochs=2)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


440/441 [============================>.] - ETA: 0s - loss: 6.6647 - recall_at_10: 0.0723 - ndcg_at_10: 0.0505 - regularization_loss: 0.0000e+00

The sampler InBatchSampler returned no samples for this batch.


441/441 [==============================] - 42s 62ms/step - loss: 6.6649 - recall_at_10: 0.0726 - ndcg_at_10: 0.0506 - regularization_loss: 0.0000e+00 - val_loss: 6.1044 - val_recall_at_10: 0.1527 - val_ndcg_at_10: 0.0800 - val_regularization_loss: 0.0000e+00
Epoch 2/2
441/441 [==============================] - 26s 58ms/step - loss: 4.4439 - recall_at_10: 0.3799 - ndcg_at_10: 0.2074 - regularization_loss: 0.0000e+00 - val_loss: 5.8538 - val_recall_at_10: 0.2354 - val_ndcg_at_10: 0.1227 - val_regularization_loss: 0.0000e+00
CPU times: user 1min 7s, sys: 12.5 s, total: 1min 19s
Wall time: 1min 10s


#### Exporting query (user) model

We export the query tower to use it later during the model deployment stage with Merlin Systems.

In [13]:
query_tower = model_tt.retrieval_block.query_block()
query_tower.save(os.path.join(BASE_DIR, "query_tower"))

INFO:tensorflow:Assets written to: /workspace/recsys_tutorial/query_tower/assets


INFO:tensorflow:Assets written to: /workspace/recsys_tutorial/query_tower/assets


### Train a ranking Model

In [14]:
schema = train.schema
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'target'

In [15]:
# Augment the batch of positive interactions with `n_per_positive` negatives sampled from the same batch. 
from merlin.models.tf.data_augmentation.negative_sampling import UniformNegativeSampling
from merlin.models.tf.dataset import BatchedDataset

batch_size, n_per_positive = 2048, 128
add_negatives = UniformNegativeSampling(schema, n_per_positive, seed=42, return_tuple=True)
dataset = BatchedDataset(train, batch_size=batch_size, shuffle=True)
dataset = dataset.map(add_negatives)

In [16]:
train.to_ddf().compute().target.value_counts()

1    3605987
Name: target, dtype: int32

In [17]:
#next(iter(dataset))

In [18]:
model = mm.DLRMModel(
    schema,
    embedding_dim=32,
    bottom_block=mm.MLPBlock([64, 32]),
    top_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

In [19]:
%%time
model.compile(optimizer='adam', run_eagerly=False, metrics=[], 
              weighted_metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]
             )
model.fit(dataset, epochs=2, train_metrics_steps=100)
#model.fit(dataset, epochs=2, class_weight = {0: 1, 1: n_per_positive}, train_metrics_steps=100)

Epoch 1/2
1761/1761 [==============================] - 203s 112ms/step - loss: 0.0299 - binary_accuracy: 0.9434 - auc: 0.9043 - regularization_loss: 0.0000e+00
Epoch 2/2
1761/1761 [==============================] - 198s 112ms/step - loss: 0.0101 - binary_accuracy: 0.9961 - auc: 0.9959 - regularization_loss: 0.0000e+00
CPU times: user 7min 57s, sys: 35.5 s, total: 8min 32s
Wall time: 6min 42s


In [20]:
valid_dataset = BatchedDataset(valid, shuffle=False, batch_size = batch_size)
valid_dataset = valid_dataset.map(add_negatives)
metrics = model.evaluate(valid_dataset, return_dict=True)
metrics

236/236 [==============================] - 22s 86ms/step - loss: 0.0193 - binary_accuracy: 0.9931 - auc: 0.9714 - regularization_loss: 0.0000e+00


{'loss': 0.019326377660036087,
 'binary_accuracy': 0.993109941482544,
 'auc': 0.9713854193687439,
 'regularization_loss': 0.0}

#### Set up a feature store with Feast

In [21]:
!rm -rf $BASE_DIR/feature_repo
!cd $BASE_DIR && feast init feature_repo

/bin/bash: feast: command not found


Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage

Creating a new Feast repository in /Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo.

You should be seeing a message like Creating a new Feast repository in ... printed out above. Now, navigate to the feature_repo folder and remove the demo parquet file created by default, and examples.py file.

In [22]:
feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
if os.path.exists(f"{feature_repo_path}/example.py"):
    os.remove(f"{feature_repo_path}/example.py")
if os.path.exists(f"{feature_repo_path}/data/driver_stats.parquet"):
    os.remove(f"{feature_repo_path}/data/driver_stats.parquet")

In [23]:
from merlin.models.utils.dataset import unique_rows_by_features

user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID)
    .compute()
    .reset_index(drop=True)
)

In [25]:
user_features.head()

,user_id,ts_hour,ts_weekday,ts_day,product_id_count
0,1,9,3,2,4.795791
1,2,6,1,26,8.750050
2,3,10,2,18,2.772589
3,4,5,6,15,7.385851
4,5,13,7,8,3.931826


In [ ]:
user_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "user_features.parquet")
)

In [ ]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
    .compute()
    .reset_index(drop=True)
)

In [ ]:
# save to disk
item_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "item_features.parquet")
)

In [ ]:
item_embs = model_tt.item_embeddings(
    Dataset(item_features, schema=schema), batch_size=1024
)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [ ]:
# select only item_id together with embedding columns
item_embeddings = item_embs_df.drop(
    columns=["item_category", "item_shop", "item_brand"]
)

In [ ]:
# save to disk
item_embeddings.to_parquet(os.path.join(BASE_DIR, "item_embeddings.parquet"))

### Summary 

In this hands-on lab we learned ...